In [12]:
import os
import json
import fitz  # PyMuPDF
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from urllib.parse import urljoin

def get_bill_text(input_pdf_path, output_json_path, title):
    # Open the original PDF and extract all text
    full_text = ""
    with fitz.open(input_pdf_path) as pdf:
        for page_num in range(len(pdf)):
            page = pdf[page_num]
            text = page.get_text()
            full_text += text + " "

    # Create a dictionary to store the data
    summary_data = {
        "title": title,
        # "summary": summary,
        "full_text": full_text
    }

    # Write the data to a JSON file
    with open(output_json_path, 'w') as json_file:
        json.dump(summary_data, json_file, indent=4)

    return os.path.abspath(output_json_path)

# Example usage
input_pdf_path = "bill_text.pdf"
output_json_path = "path_to_output_json.json"
title = "Bill Summary"

get_bill_text(input_pdf_path, output_json_path, title)

def download_pdf(pdf_url, local_path="bill_text.pdf"):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(local_path, 'wb') as file:
            file.write(response.content)
        return local_path
    else:
        raise Exception(f"Failed to download PDF from {pdf_url}")


In [13]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def fetch_bill_details(bill_page_url):
    """
    Fetches details of a bill from the Florida Senate Bill page and downloads its PDF.
    :param bill_page_url: URL of the specific bill page.
    :return: A dictionary containing the bill title, description, local PDF path, and full_text.
    """
    base_url = 'https://www.flsenate.gov'
    response = requests.get(urljoin(base_url, bill_page_url))

    bill_details = {
        "title": "",
        "description": "",
        "pdf_path": "",  # Changed from pdf_url to pdf_path
        "full_text": ""  # Include full_text
    }

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the bill title
        bill_title_tag = soup.find('div', id='prevNextBillNav').find_next('h2')
        if bill_title_tag:
            bill_details["title"] = bill_title_tag.get_text(strip=True)

        # Extract the bill description
        bill_description_tag = soup.find('p', class_='width80')
        if bill_description_tag:
            bill_details["description"] = bill_description_tag.get_text(strip=True)

        # Extract the bill PDF link and download it
        bill_pdf_link = soup.find('a', class_='lnk_BillTextPDF')
        if bill_pdf_link:
            pdf_url = urljoin(base_url, bill_pdf_link['href'])
            bill_details["pdf_path"] = download_pdf(pdf_url)

        # Extract and store the full_text
        full_text = extract_full_text_from_pdf(bill_details["pdf_path"])
        bill_details["full_text"] = full_text

    return bill_details

bill_details = fetch_bill_details("https://www.flsenate.gov/Session/Bill/2023/23/ByCategory/?Tab=BillText")


NameError: name 'extract_full_text_from_pdf' is not defined

In [9]:
import os
import json
import fitz  # PyMuPDF
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

def get_bill_text(input_pdf_path, output_json_path, title):
    # Open the original PDF and extract all text
    full_text = ""
    with fitz.open(input_pdf_path) as pdf:
        for page_num in range(len(pdf)):
            page = pdf[page_num]
            text = page.get_text()
            full_text += text + " "

    # Create a dictionary to store the data
    summary_data = {
        "title": title,
        # "summary": summary,
        "full_text": full_text
    }

    # Write the data to a JSON file
    with open(output_json_path, 'w') as json_file:
        json.dump(summary_data, json_file, indent=4)

    return os.path.abspath(output_json_path)

def download_pdf(pdf_url, local_path="bill_text.pdf"):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(local_path, 'wb') as file:
            file.write(response.content)
        return local_path
    else:
        raise Exception(f"Failed to download PDF from {pdf_url}")

def fetch_bill_details(bill_page_url):
    """
    Fetches details of a bill from the Florida Senate Bill page and downloads its PDF.
    :param bill_page_url: URL of the specific bill page.
    :return: A dictionary containing the bill title, description, local PDF path, and full_text.
    """
    base_url = 'https://www.flsenate.gov'
    response = requests.get(urljoin(base_url, bill_page_url))

    bill_details = {
        "title": "",
        "description": "",
        "pdf_path": "",  # Changed from pdf_url to pdf_path
        "full_text": ""  # Include full_text
    }

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the bill title
        bill_title_tag = soup.find('div', id='prevNextBillNav').find_next('h2')
        if bill_title_tag:
            bill_details["title"] = bill_title_tag.get_text(strip=True)

        # Extract the bill description
        bill_description_tag = soup.find('p', class_='width80')
        if bill_description_tag:
            bill_details["description"] = bill_description_tag.get_text(strip=True)

        # Extract the bill PDF link and download it
        bill_pdf_link = soup.find('a', class_='lnk_BillTextPDF')
        if bill_pdf_link:
            pdf_url = urljoin(base_url, bill_pdf_link['href'])
            bill_details["pdf_path"] = download_pdf(pdf_url)

        # Call get_bill_text to set the "full_text" field
        get_bill_text(bill_details["pdf_path"], "output.json", bill_details["title"])

    return bill_details

bill_details = fetch_bill_details("https://www.flsenate.gov/Session/Bill/2023/23/ByCategory/?Tab=BillText")


In [11]:
import json
import boto3

def upload_to_s3(bucket_name, bill_details):
    """
    Uploads bill details and the associated PDF to an S3 bucket with public read access.

    :param bucket_name: The name of the S3 bucket.
    :param bill_details: A dictionary containing the bill's details, including the local PDF path.
    """
    s3_client = boto3.client('s3')

    # Upload the PDF file with public read access
    pdf_path = bill_details["pdf_path"]
    pdf_key = f"pdfs/{pdf_path.split('/')[-1]}"
    with open(pdf_path, 'rb') as pdf_file:
        s3_client.upload_fileobj(
            pdf_file,
            bucket_name,
            pdf_key,
            ExtraArgs={'ACL': 'public-read'}  # Set ACL to 'public-read'
        )

    # Prepare and upload the bill details as a JSON file with public read access
    json_details = bill_details.copy()
    json_details["pdf_path"] = f"s3://{bucket_name}/{pdf_key}"  # Update the PDF path to the S3 location
    json_key = f"bill_details/{pdf_path.split('/')[-1].replace('.pdf', '.json')}"
    s3_client.put_object(
        Body=json.dumps(json_details),
        Bucket=bucket_name,
        Key=json_key,
        ACL='public-read'  # Set ACL to 'public-read'
    )

    return {
        "pdf_s3_path": json_details["pdf_path"],
        "json_s3_path": f"s3://{bucket_name}/{json_key}"
    }

# Example usage
bucket_name = 'ddp-bills'
bill_page_url = "https://www.flsenate.gov/Session/Bill/2023/23/ByCategory/?Tab=BillText"
bill_details = fetch_bill_details(bill_page_url)
s3_paths = upload_to_s3(bucket_name, bill_details)
print(s3_paths)


from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Define the SQLAlchemy model for the 'bill' table
Base = declarative_base()
class Bill(Base):
    __tablename__ = 'bill'

    id = Column(Integer, primary_key=True)
    govId = Column(String)
    billTextPath = Column(String)

def connect_to_db(host, database, user, password, port=3306):
    """
    Establishes a connection to the MariaDB database and returns a session.
    """
    try:
        # Create a SQLAlchemy database engine with host and port
        db_url = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
        engine = create_engine(db_url, echo=True)  # Set echo to True for debugging

        # Create a session
        Session = sessionmaker(bind=engine)
        session = Session()

        print("Connected to MariaDB database")
        return session
    except Exception as e:
        print(f"Error: {e}")
        return None

def insert_bill(session, govId, billTextPath):
    """
    Inserts a new bill into the 'bill' table using SQLAlchemy.
    """
    try:
        # Create a new Bill instance
        new_bill = Bill(govId=govId, billTextPath=billTextPath)

        # Add the new_bill to the session and commit it
        session.add(new_bill)
        session.commit()

        print("Bill inserted successfully.")
        return new_bill.id
    except Exception as e:
        print(f"Error: {e}")
        return None

# Database connection details (including the default port 3306)
db_host = 'ddp-api.czqcac8oivov.us-east-1.rds.amazonaws.com'
db_name = 'digital_democracy'
db_user = 'DataWithAlex'
db_password = '%Mineguy29'  # Replace with your actual database password
db_port = 3306  # Specify the port if needed, otherwise omit this line

# Connect to the database
session = connect_to_db(db_host, db_name, db_user, db_password, db_port)

if session:
    # Example bill details
    bill_page_url = "https://www.flsenate.gov/Session/Bill/2023/23/ByCategory/?Tab=BillText"
    bill_details = fetch_bill_details(bill_page_url)
    s3_paths = upload_to_s3('ddp-bills', bill_details)

    # Insert the bill into the database
    bill_id = insert_bill(session, "test", s3_paths["pdf_s3_path"])

    # Close the database session
    session.close()
    print("Database session closed.")
else:
    print("Database connection not established.")

{'pdf_s3_path': 's3://ddp-bills/pdfs/bill_text.pdf', 'json_s3_path': 's3://ddp-bills/bill_details/bill_text.json'}
Connected to MariaDB database
2024-01-22 16:11:52,558 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-01-22 16:11:52,559 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:11:53,039 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-01-22 16:11:53,040 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:11:53,201 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-01-22 16:11:53,201 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:11:53,466 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-22 16:11:53,591 INFO sqlalchemy.engine.Engine INSERT INTO bill (`govId`, `billTextPath`) VALUES (%(govId)s, %(billTextPath)s)
2024-01-22 16:11:53,592 INFO sqlalchemy.engine.Engine [generated in 0.12479s] {'govId': 'test', 'billTextPath': 's3://ddp-bills/pdfs/bill_text.pdf'}
2024-01-22 16:11:53,994 INFO sqlalchemy.engine.Engine COMMIT
Bill inse

In [4]:
# %pip install reportlab
%pip install PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 16.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 26.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import json
import fitz  # PyMuPDF
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

def get_bill_text(input_pdf_path, output_json_path, title):
    # Open the original PDF and extract all text
    full_text = ""
    with fitz.open(input_pdf_path) as pdf:
        for page_num in range(len(pdf)):
            page = pdf[page_num]
            text = page.get_text()
            full_text += text + " "

    # Create a dictionary to store the data
    summary_data = {
        "title": title,
        # "summary": summary,
        "full_text": full_text
    }

    # Write the data to a JSON file
    with open(output_json_path, 'w') as json_file:
        json.dump(summary_data, json_file, indent=4)

    return os.path.abspath(output_json_path)

def download_pdf(pdf_url, local_path="bill_text.pdf"):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(local_path, 'wb') as file:
            file.write(response.content)
        return local_path
    else:
        raise Exception(f"Failed to download PDF from {pdf_url}")

def fetch_bill_details(bill_page_url):
    """
    Fetches details of a bill from the Florida Senate Bill page and downloads its PDF.
    :param bill_page_url: URL of the specific bill page.
    :return: A dictionary containing the bill title, description, local PDF path, and full_text.
    """
    base_url = 'https://www.flsenate.gov'
    response = requests.get(urljoin(base_url, bill_page_url))

    bill_details = {
        "title": "",
        "description": "",
        "pdf_path": "",  # Changed from pdf_url to pdf_path
        "full_text": ""  # Include full_text
    }

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the bill title
        bill_title_tag = soup.find('div', id='prevNextBillNav').find_next('h2')
        if bill_title_tag:
            bill_details["title"] = bill_title_tag.get_text(strip=True)

        # Extract the bill description
        bill_description_tag = soup.find('p', class_='width80')
        if bill_description_tag:
            bill_details["description"] = bill_description_tag.get_text(strip=True)

        # Extract the bill PDF link and download it
        bill_pdf_link = soup.find('a', class_='lnk_BillTextPDF')
        if bill_pdf_link:
            pdf_url = urljoin(base_url, bill_pdf_link['href'])
            bill_details["pdf_path"] = download_pdf(pdf_url)

        # Call get_bill_text to set the "full_text" field
        json_path = get_bill_text(bill_details["pdf_path"], "output.json", bill_details["title"])
        bill_details["billTextPath"] = json_path  # Add the Object URL to billTextPath

    return bill_details

bill_details = fetch_bill_details("https://www.flsenate.gov/Session/Bill/2023/23/ByCategory/?Tab=BillText")


In [16]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Define the SQLAlchemy model for the 'bill' table
Base = declarative_base()
class Bill(Base):
    __tablename__ = 'bill'

    id = Column(Integer, primary_key=True)
    govId = Column(String)
    billTextPath = Column(String)

def connect_to_db(host, database, user, password, port=3306):
    """
    Establishes a connection to the MariaDB database and returns a session.
    """
    try:
        # Create a SQLAlchemy database engine with host and port
        db_url = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
        engine = create_engine(db_url, echo=True)  # Set echo to True for debugging

        # Create a session
        Session = sessionmaker(bind=engine)
        session = Session()

        print("Connected to MariaDB database")
        return session
    except Exception as e:
        print(f"Error: {e}")
        return None

def insert_bill(session, govId, billTextPath):
    """
    Inserts a new bill into the 'bill' table using SQLAlchemy.
    """
    try:
        # Create a new Bill instance with the S3 URL
        new_bill = Bill(govId=govId, billTextPath=billTextPath)

        # Add the new_bill to the session and commit it
        session.add(new_bill)
        session.commit()

        print("Bill inserted successfully.")
        return new_bill.id
    except Exception as e:
        print(f"Error: {e}")
        return None

# Database connection details (including the default port 3306)
db_host = 'ddp-api.czqcac8oivov.us-east-1.rds.amazonaws.com'
db_name = 'digital_democracy'
db_user = 'DataWithAlex'
db_password = '%Mineguy29'  # Replace with your actual database password
db_port = 3306  # Specify the port if needed, otherwise omit this line

# Connect to the database
session = connect_to_db(db_host, db_name, db_user, db_password, db_port)

if session:
    # Example bill details
    bill_page_url = "https://www.flsenate.gov/Session/Bill/2023/23/ByCategory/?Tab=BillText"
    bill_details = fetch_bill_details(bill_page_url)
    
    # Construct the S3 URL for billTextPath
    s3_url = f"https://ddp-bills.s3.amazonaws.com/bill_details/{bill_details['pdf_path'].split('/')[-1].replace('.pdf', '.json')}"
    
    # Insert the bill into the database with the S3 URL as billTextPath
    bill_id = insert_bill(session, "test", s3_url)

    # Close the database session
    session.close()
    print("Database session closed.")
else:
    print("Database connection not established.")


Connected to MariaDB database
2024-01-22 16:20:54,907 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-01-22 16:20:54,908 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:20:55,253 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-01-22 16:20:55,254 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:20:55,385 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-01-22 16:20:55,387 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:20:55,480 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-22 16:20:55,630 INFO sqlalchemy.engine.Engine INSERT INTO bill (`govId`, `billTextPath`) VALUES (%(govId)s, %(billTextPath)s)
2024-01-22 16:20:55,632 INFO sqlalchemy.engine.Engine [generated in 0.14918s] {'govId': 'test', 'billTextPath': 'https://ddp-bills.s3.amazonaws.com/bill_details/bill_text.json'}
2024-01-22 16:20:55,815 INFO sqlalchemy.engine.Engine COMMIT
Bill inserted successfully.
2024-01-22 16:20:55,909 INFO sqlalchemy.engine.Engine BEGIN (implic